# Visualizing Word Embeddings on the Tensorboard

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import zipfile
from tensorboard.plugins import projector
import csv

## Read the GloVe file

Here we first need to download the GloVe word embeddings (`glove.6B.zip`) found at this [website](https://nlp.stanford.edu/projects/glove/). Then we read the GloVe file to get the first 50000 words in the file. We will be using 50 dimensional word vectors

In [5]:
vocabulary_size = 50000

embedding_df = [] 
index = []
# Open the zip file
with zipfile.ZipFile('glove.6B.zip') as glovezip:
    # Read the file with 50 dimensional embeddings
    with glovezip.open('glove.6B.50d.txt') as glovefile:
        # Read line by line
        for li, line in enumerate(glovefile):
            # Print progress
            if (li+1)%10000==0: print('.',end='')
                
            # Get the word and the corresponding vector
            line_tokens = line.decode('utf-8').split(' ')
            word = line_tokens[0]
            vector = [float(v) for v in line_tokens[1:]]
            
            assert len(vector)==50
            index.append(word)
            # Update the embedding matrix
            embedding_df.append(np.array(vector))
            
            # If the first 50000 words being read, finish
            if li >= vocabulary_size-1:
                break

embedding_df = pd.DataFrame(embedding_df, index=index)
embedding_df.head(n=10)

.....

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
the,0.418000,0.249680,-0.41242,0.121700,0.345270,-0.044457,-0.49688,-0.178620,-0.000660,-0.656600,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.442510,0.187850,0.002785,-0.184110,-0.115140,-0.785810
",",0.013441,0.236820,-0.16899,0.409510,0.638120,0.477090,-0.42852,-0.556410,-0.364000,-0.239380,...,-0.080262,0.630030,0.321110,-0.467650,0.227860,0.360340,-0.378180,-0.566570,0.044691,0.303920
.,0.151640,0.301770,-0.16763,0.176840,0.317190,0.339730,-0.43478,-0.310860,-0.449990,-0.294860,...,-0.000064,0.068987,0.087939,-0.102850,-0.139310,0.223140,-0.080803,-0.356520,0.016413,0.102160
of,0.708530,0.570880,-0.47160,0.180480,0.544490,0.726030,0.18157,-0.523930,0.103810,-0.175660,...,-0.347270,0.284830,0.075693,-0.062178,-0.389880,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
to,0.680470,-0.039263,0.30186,-0.177920,0.429620,0.032246,-0.41376,0.132280,-0.298470,-0.085253,...,-0.094375,0.018324,0.210480,-0.030880,-0.197220,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
and,0.268180,0.143460,-0.27877,0.016257,0.113840,0.699230,-0.51332,-0.473680,-0.330750,-0.138340,...,-0.069043,0.368850,0.251680,-0.245170,0.253810,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
in,0.330420,0.249950,-0.60874,0.109230,0.036372,0.151000,-0.55083,-0.074239,-0.092307,-0.328210,...,-0.486090,-0.008027,0.031184,-0.365760,-0.426990,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
a,0.217050,0.465150,-0.46757,0.100820,1.013500,0.748450,-0.53104,-0.262560,0.168120,0.131820,...,0.138130,0.369730,-0.642890,0.024142,-0.039315,-0.260370,0.120170,-0.043782,0.410130,0.179600
"""",0.257690,0.456290,-0.76974,-0.376790,0.592720,-0.063527,0.20545,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650
's,0.237270,0.404780,-0.20547,0.588050,0.655330,0.328670,-0.81964,-0.232360,0.274280,0.242650,...,-0.123420,0.659610,-0.518020,-0.829950,-0.082739,0.281550,-0.423000,-0.273780,-0.007901,-0.030231


## Create TensorFlow Variable and config

Here we create a TensorFlow variable to store the embeddings we read above and save it to the disk. This is necessary for the visualization. Along with that we will save metadata, which has the labels for each embedding.

In [6]:
# Create a directory to save our model
log_dir = 'embeddings'
os.makedirs(log_dir, exist_ok=True)

# Save the weights we want to analyse as a variable. 
embeddings = tf.Variable(embedding_df.values)
print(f"weights.shape: {embeddings.shape}")

# Create a checkpoint from embedding
checkpoint = tf.train.Checkpoint(embedding=embeddings)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

with open(os.path.join(log_dir, 'metadata.tsv'), 'w', encoding='utf-8') as f:
    for w in embedding_df.index:
        f.write(w+'\n')

weights.shape: (50000, 50)


## Running the TensorBoard

* cd into the `packt_nlp_tf2/Appendix` directory
* Run `tensorboard --logdir embeddings`

## Visualizing the embeddings

In [4]:
config = projector.ProjectorConfig()

# You can add multiple embeddings. Here we add only one.
embedding_config = config.embeddings.add()
embedding_config.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# Link this tensor to its metadata file (e.g. labels).
embedding_config.metadata_path = 'metadata.tsv'

# TensorBoard will read this file during startup.
projector.visualize_embeddings(log_dir, config)